In [27]:
import tensorflow as tf  # Mengimpor modul TensorFlow untuk digunakan dalam kode.
import numpy as np  # Mengimpor modul NumPy dengan alias np untuk digunakan dalam kode.
import os  # Mengimpor modul os untuk berinteraksi dengan sistem operasi seperti path, direktori, dan lainnya.
import time  # Mengimpor modul time untuk bekerja dengan waktu dalam kode.


In [28]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
# Mendapatkan file 'shakespeare.txt' dari URL yang disediakan menggunakan fungsi get_file dari modul keras.utils di TensorFlow.

In [29]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# Membaca file 'shakespeare.txt' dengan mode 'rb' (read binary) kemudian melakukan decoding teks menggunakan utf-8.

print(f'Length of text: {len(text)} characters')
# Menampilkan panjang teks dalam karakter dari file yang telah dibaca.

Length of text: 1115394 characters


In [30]:
print(text[:250])
# Menampilkan 250 karakter pertama dari teks yang telah dibaca dari file 'shakespeare.txt'.

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [31]:
vocab = sorted(set(text))
# Membuat himpunan karakter unik (vocab) dari teks yang telah dibaca, lalu diurutkan.

print(f'{len(vocab)} unique characters')
# Menampilkan jumlah karakter unik dalam teks.

65 unique characters


In [32]:
example_texts = ['abcdefg', 'xyz']
# Mendefinisikan beberapa teks contoh dalam sebuah list.

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
# Membagi teks-teks contoh menjadi karakter-karakter menggunakan fungsi unicode_split dari TensorFlow,
# dengan encoding input yang ditentukan sebagai UTF-8.

In [33]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)
# Membuat layer StringLookup dari TensorFlow untuk mengonversi karakter ke ID menggunakan daftar karakter unik (vocab)
# tanpa menggunakan mask_token.

In [34]:
ids = ids_from_chars(chars)
# Menggunakan layer StringLookup yang telah dibuat sebelumnya untuk mengonversi karakter-karakter menjadi ID.
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [35]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
# Membuat layer StringLookup baru untuk mengonversi ID kembali ke karakter dengan menggunakan vocabulary
# yang sama dengan yang digunakan sebelumnya, dengan parameter invert=True.

In [36]:
chars = chars_from_ids(ids)
# Menggunakan layer StringLookup yang telah dibuat sebelumnya untuk mengonversi ID kembali menjadi karakter.
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [37]:
tf.strings.reduce_join(chars, axis=-1).numpy()
# Menggunakan fungsi reduce_join dari TensorFlow untuk menggabungkan karakter-karakter menjadi string pada axis -1 (axis terakhir) dan mengonversi hasilnya menjadi array NumPy.

array([b'abcdefg', b'xyz'], dtype=object)

In [38]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)
# Membuat fungsi text_from_ids yang menggunakan chars_from_ids untuk mengonversi IDs kembali menjadi teks.

In [39]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
# Menggunakan ids_from_chars untuk mengonversi seluruh teks menjadi IDs dengan membagi teks menjadi karakter menggunakan UTF-8.
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [40]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
# Membuat objek Dataset dari tensor all_ids menggunakan from_tensor_slices dari TensorFlow.

In [41]:
for ids in ids_dataset.take(10):
    # Mengambil 10 elemen pertama dari ids_dataset dan mengiterasinya.

    print(chars_from_ids(ids).numpy().decode('utf-8'))
    # Mengonversi IDs kembali menjadi karakter menggunakan chars_from_ids,
    # lalu didecode menjadi string dengan utf-8 dan mencetak hasilnya.

F
i
r
s
t
 
C
i
t
i


In [42]:
seq_length = 100
# Menetapkan panjang urutan (sequence length) sebesar 100.

In [43]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)
# Membagi dataset ids menjadi urutan-urutan dengan panjang seq_length+1 dan mengabaikan bagian sisa.

for seq in sequences.take(1):
    print(chars_from_ids(seq))
# Mengambil 1 urutan pertama dari dataset sequences dan mengonversi IDs kembali menjadi karakter.

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [44]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())
# Mengambil 5 urutan pertama dari dataset sequences, mengonversi IDs kembali menjadi teks, dan mencetak hasilnya dalam bentuk array NumPy.

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [45]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text
# Membuat fungsi split_input_target untuk membagi urutan menjadi input dan target dengan menggeser satu karakter.

In [46]:
split_input_target(list("Tensorflow"))
# Menggunakan fungsi split_input_target untuk membagi urutan karakter "Tensorflow" menjadi input dan target.

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [47]:
dataset = sequences.map(split_input_target)
# Menggunakan fungsi map untuk menerapkan fungsi split_input_target ke setiap elemen dalam dataset sequences.

In [48]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())
# Mengambil 1 contoh input dan target dari dataset yang telah dimodifikasi, mengonversi mereka kembali menjadi teks, dan mencetaknya.

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [49]:
BATCH_SIZE = 64
# Menetapkan ukuran batch sebesar 64.

BUFFER_SIZE = 10000
# Menetapkan ukuran buffer untuk melakukan pengacakan dataset.

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))
# Melakukan pengacakan, pembagian menjadi batch, dan prefetching untuk meningkatkan performa dataset.
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [50]:
vocab_size = len(ids_from_chars.get_vocabulary())
# Mendapatkan panjang vocabulary yang digunakan dalam layer StringLookup.

embedding_dim = 256
# Menetapkan dimensi embedding sebesar 256.

rnn_units = 1024
# Menetapkan jumlah unit dalam RNN sebesar 1024.

In [51]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        # Mendefinisikan model dengan layer-layer yang dibutuhkan untuk membangun arsitektur model.

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        # Layer embedding untuk mengubah ID-token menjadi representasi vektor.

        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state=True)
        # Layer GRU (Gated Recurrent Unit) sebagai bagian dari model untuk menangani urutan data dengan kemampuan mengembalikan urutan dan state.

        self.dense = tf.keras.layers.Dense(vocab_size)
        # Layer Dense untuk melakukan output prediksi pada setiap timestamp.

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        # Memasukkan input ke dalam layer embedding untuk mendapatkan representasi vektor.

        if states is None:
            states = self.gru.get_initial_state(x)
        # Menginisialisasi states jika tidak ada states yang diberikan sebagai input.

        x, states = self.gru(x, initial_state=states, training=training)
        # Memasukkan input ke dalam layer GRU untuk memperoleh output dan states.

        x = self.dense(x, training=training)
        # Memasukkan output dari GRU ke dalam layer Dense untuk mendapatkan hasil prediksi.

        if return_state:
            return x, states
        else:
            return x
        # Mengembalikan hasil prediksi atau hasil dan states tergantung pada nilai return_state.

In [52]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)
# Membuat objek model menggunakan class MyModel yang telah dibuat sebelumnya dengan parameter yang telah ditentukan sebelumnya.

In [53]:
# uji model

for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    # Memperoleh prediksi dari contoh batch input menggunakan model.

    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")
    # Menampilkan bentuk (shape) dari prediksi yang dihasilkan oleh model.
    # (batch_size, sequence_length, vocab_size) menunjukkan bahwa model mengeluarkan prediksi untuk setiap urutan pada setiap batch dengan dimensi vocab_size.


(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [54]:
model.summary()
# Menampilkan ringkasan (summary) dari arsitektur model yang telah dibuat, menunjukkan informasi tentang layer dan jumlah parameter yang dapat disesuaikan.

Model: "my_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     multiple                  16896     
                                                                 
 gru_1 (GRU)                 multiple                  3938304   
                                                                 
 dense_1 (Dense)             multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [55]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
# Menggunakan fungsi tf.random.categorical untuk mengambil sampel indeks dari distribusi probabilitas dalam example_batch_predictions.

sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
# Menghilangkan dimensi yang tidak perlu dengan menggunakan tf.squeeze, lalu mengonversi hasilnya menjadi array NumPy.

In [56]:
sampled_indices

array([17,  5, 17, 54, 15, 21, 45, 29, 28, 36, 63, 33, 27, 27, 47, 39,  9,
       46, 58, 21, 43, 55, 62, 38, 33,  3, 63, 53, 40, 61, 43, 45, 27, 55,
       39, 23, 19, 57, 11, 27, 54, 10, 29,  0, 52, 13, 44, 41, 54, 64, 18,
       63, 27, 50, 65, 14, 55,  8, 37, 56, 21, 15, 34, 15, 18, 59, 25, 41,
       62, 26, 34, 28,  8, 65, 52, 24, 40, 35, 36, 25, 59, 47, 22, 19, 41,
       34, 51, 62, 61,  3, 27, 31, 13, 37,  2, 53, 12, 30, 34, 60])

In [58]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
# Mencetak contoh input dari batch pertama sebagai teks.

print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())
# Mencetak prediksi karakter berikutnya berdasarkan sampel indeks yang diambil dari model sebagai teks.

Input:
 b'd you that do abet him in this kind\nCherish rebellion and are rebels all.\n\nNORTHUMBERLAND:\nThe noble'

Next Char Predictions:
 b'D&DoBHfPOWxTNNhZ.gsHdpwYT!xnavdfNpZJFr:No3P[UNK]m?eboyExNkzAp-XqHBUBEtLbwMUO-zmKaVWLthIFbUlwv!NR?X n;QUu'


In [59]:
# train model

loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
# Mendefinisikan fungsi loss menggunakan SparseCategoricalCrossentropy untuk menghitung nilai loss pada model yang mengeluarkan logits.


In [60]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
# Menghitung rata-rata loss dengan menggunakan fungsi loss pada target dan prediksi yang dihasilkan oleh model.

print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
# Menampilkan bentuk (shape) dari prediksi yang dihasilkan oleh model.

print("Mean loss:        ", example_batch_mean_loss)
# Menampilkan nilai rata-rata loss pada contoh batch.


Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.189529, shape=(), dtype=float32)


In [61]:
tf.exp(example_batch_mean_loss).numpy()
# Menghitung eksponen dari nilai rata-rata loss menggunakan fungsi tf.exp dan mengonversi hasilnya menjadi array NumPy.

model.compile(optimizer='adam', loss=loss)
# Mengompilasi model dengan optimizer 'adam' dan fungsi loss yang telah ditentukan sebelumnya.

# Direktori tempat checkpoints akan disimpan.
checkpoint_dir = './training_checkpoints'
# Nama file checkpoint.
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)
# Membuat callback ModelCheckpoint untuk menyimpan bobot (weights) model selama pelatihan.


In [62]:
# Lakukan Proses Training

EPOCHS = 20
# Menetapkan jumlah epochs sebesar 20.

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])
# Melatih model menggunakan dataset yang telah dipersiapkan sebelumnya selama jumlah epochs yang telah ditentukan,
# dengan menggunakan callback ModelCheckpoint untuk menyimpan bobot model pada setiap epoch.


Epoch 1/20
172/172 [==============================] - 15s 59ms/step - loss: 2.7001
Epoch 2/20
172/172 [==============================] - 12s 58ms/step - loss: 1.9753
Epoch 3/20
172/172 [==============================] - 12s 61ms/step - loss: 1.6936
Epoch 4/20
172/172 [==============================] - 12s 61ms/step - loss: 1.5357
Epoch 5/20
172/172 [==============================] - 12s 60ms/step - loss: 1.4398
Epoch 6/20
172/172 [==============================] - 12s 63ms/step - loss: 1.3735
Epoch 7/20
172/172 [==============================] - 12s 60ms/step - loss: 1.3206
Epoch 8/20
172/172 [==============================] - 12s 58ms/step - loss: 1.2756
Epoch 9/20
172/172 [==============================] - 13s 61ms/step - loss: 1.2355
Epoch 10/20
172/172 [==============================] - 13s 61ms/step - loss: 1.1941
Epoch 11/20
172/172 [==============================] - 13s 62ms/step - loss: 1.1542
Epoch 12/20
172/172 [==============================] - 12s 61ms/step - loss: 1.1122
E

In [63]:
# Generate Teks

class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # Create a mask to prevent "[UNK]" from being generated.
        skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index.
            values=[-float('inf')] * len(skip_ids),
            indices=skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())])
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = self.model(inputs=input_ids, states=states,
                                              return_state=True)
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits / self.temperature
        # Apply the prediction mask: prevent "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states


In [64]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)
# Membuat objek OneStep yang menggunakan model, fungsi untuk mengonversi karakter ke ID, dan fungsi untuk mengonversi ID ke karakter.

start = time.time()
# Membuat variabel start untuk mengukur waktu mulai.

states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]
# Memulai dengan teks 'ROMEO:' dan menyimpannya dalam variabel next_char, serta membuat list result untuk menyimpan karakter-karakter yang akan di-generate.

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)
# Menggunakan model OneStep untuk meng-generate 1000 karakter berikutnya dari teks yang telah diberikan.

result = tf.strings.join(result)
end = time.time()
# Menggabungkan hasil karakter yang di-generate menjadi sebuah teks dan mengukur waktu selesai.

print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)
# Mencetak hasil teks yang di-generate dan waktu yang dibutuhkan untuk proses generasi karakter.


ROMEO:
When I was born.
Go you to pluen here! Mark a corn to rue.

DUKE VINCENTIO:
Are you more, by God's grave, be not again:
Degree is the clamour of one father.
But what may intended but a father's
that methinks up this hour with her kings.

QUEEN ELIZABETH:
Come yo in cold and patrician,
Pronounce is but rotten person.

MIRANDA:
Sir baid, your drum,
And pluch his rope more pleasant gate of Nobforch,
I'll hear no more: she does be sud
And scorn to seek him help, help you,
And give him limber, shall get thee to your horse:
Is Romeo when it speak of wine.

MONTAGUE:
And leave me, sir; for of his order father,
Your worthy garments are.

ROMEO:
Sir, my mother!' mind! canst give your leave!

KING RICHARD II:
Were you the mutiners, nor the heavens
one--to greeting. Now, if an unmaid ask,
That when she see our lasts with your ancestor.

FRIAR LAURENCE:
What will they say? must he good?
This is the selfsame ways.

CLARENCE:
O, 'tis the fearful swords of Clarectes.

ANGELO:
Speed how you in 

In [65]:
# eksplor model generator

tf.saved_model.save(one_step_model, 'one_step')
# Menyimpan model OneStep ke dalam direktori 'one_step'.

one_step_reloaded = tf.saved_model.load('one_step')
# Memuat kembali model yang telah disimpan.

states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]
# Memulai dengan teks 'ROMEO:' dan membuat list result untuk menyimpan karakter-karakter yang akan di-generate.

for n in range(100):
    next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
    result.append(next_char)
# Menggunakan model yang telah dimuat kembali untuk meng-generate 100 karakter berikutnya dari teks yang telah diberikan.

print(tf.strings.join(result)[0].numpy().decode("utf-8"))
# Mencetak hasil teks yang di-generate.


ROMEO:
See, such a lodden, let a groan father sin:
Does cot on church, and if it let my head;
Which on him


# Tugas

In [66]:
class CustomTraining(MyModel):
    @tf.function
    def train_step(self, inputs):
        inputs, labels = inputs
        # Memisahkan input dan label dari data yang diberikan.

        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            # Memperoleh prediksi dari model pada mode pelatihan.

            loss = self.loss(labels, predictions)
            # Menghitung loss antara label dan prediksi.

        grads = tape.gradient(loss, model.trainable_variables)
        # Menghitung gradien loss terhadap parameter yang dapat diubah (trainable variables) dalam model.

        self.optimizer.apply_gradients(zip(grads, model.trainable_variables))
        # Menggunakan optimizer untuk menerapkan gradien ke parameter dalam model.

        return {'loss': loss}
        # Mengembalikan nilai loss dari iterasi pelatihan.

In [67]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)
# Membuat objek model CustomTraining dengan parameter yang telah ditentukan sebelumnya.

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))
# Mengompilasi model dengan optimizer Adam dan fungsi loss SparseCategoricalCrossentropy.

model.fit(dataset, epochs=1)
# Melatih model menggunakan dataset yang telah dipersiapkan sebelumnya selama 1 epoch.


172/172 [==============================] - 19s 62ms/step - loss: 2.7187


In [68]:
EPOCHS = 10
# Menetapkan jumlah epochs sebesar 10.

mean = tf.metrics.Mean()
# Membuat objek mean dari kelas tf.metrics.Mean untuk menghitung rata-rata.

for epoch in range(EPOCHS):
    start = time.time()
    # Memulai penghitungan waktu untuk setiap epoch.

    mean.reset_states()
    # Mengatur ulang nilai state pada objek mean untuk setiap epoch.

    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        # Melakukan satu iterasi pelatihan (train step) menggunakan model pada dataset.

        mean.update_state(logs['loss'])
        # Memperbarui nilai rata-rata loss dengan loss dari iterasi terkini.

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)
            # Mencetak informasi loss setiap 50 batch.

    # Menyimpan model setiap 5 epochs.
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))
# Menyimpan bobot (weights) model setelah semua epochs selesai.


Epoch 1 Batch 0 Loss 2.1647
Epoch 1 Batch 50 Loss 2.0687
Epoch 1 Batch 100 Loss 1.9699
Epoch 1 Batch 150 Loss 1.8768

Epoch 1 Loss: 1.9866
Time taken for 1 epoch 14.18 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.7966
Epoch 2 Batch 50 Loss 1.7216
Epoch 2 Batch 100 Loss 1.6927
Epoch 2 Batch 150 Loss 1.6508

Epoch 2 Loss: 1.7083
Time taken for 1 epoch 11.44 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.5957
Epoch 3 Batch 50 Loss 1.5514
Epoch 3 Batch 100 Loss 1.5673
Epoch 3 Batch 150 Loss 1.4790

Epoch 3 Loss: 1.5487
Time taken for 1 epoch 12.49 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.4589
Epoch 4 Batch 50 Loss 1.4354
Epoch 4 Batch 100 Loss 1.4370
Epoch 4 Batch 150 Loss 1.4193

Epoch 4 Loss: 1.4503
Time taken for 1 epoch 12.73 sec
_____________________________________________________________________

Terdapat beberapa perbedaan antara kode tugas dengan kode pada praktikum 2:

1. Penyesuaian Model:

- Praktikum 2: Pada praktikum 2, kode mendefinisikan dan menggunakan model recurrent neural network (RNN) sederhana untuk menghasilkan teks yang mirip dengan teks latihan.
- Tugas: Kode terlihat lebih kompleks dan merupakan pengembangan lanjutan dari model RNN yang disesuaikan dengan kebutuhan tugas tertentu. Ini mencakup penggunaan kelas CustomTraining yang merupakan turunan dari model yang sudah ada (MyModel), dengan fitur-fitur tambahan yang diintegrasikan seperti fungsi train_step.

2. Pelatihan Model:

- Praktikum 2: Praktikum 2 lebih fokus pada generasi teks dan cara mengatur dataset serta melatih model RNN untuk menghasilkan teks yang mirip.
- Tugas: Kode melibatkan proses pelatihan yang lebih terperinci dengan pembuatan loop pelatihan yang menghitung loss, melakukan optimisasi, dan mencetak informasi loss serta waktu pelatihan untuk setiap epoch.

3. Penanganan Checkpoint:

- Praktikum 2: Praktikum 2 mungkin tidak secara eksplisit menangani penyimpanan checkpoint (titik kontrol) model untuk digunakan di masa depan.
- Tugas: terdapat logika yang menyimpan bobot (weights) model setiap 5 epochs menggunakan model.save_weights, yang merupakan praktek umum untuk menyimpan kemajuan pelatihan model secara teratur untuk penggunaan selanjutnya.

4. Pengukuran Metrik:

- Praktikum 2: Pada praktikum 2, pengukuran metrik (seperti loss) mungkin tidak disimpan atau dicatat secara eksplisit.
- Tugas: objek tf.metrics.Mean() untuk menghitung rata-rata loss selama pelatihan. Hal ini membantu dalam melacak dan melaporkan rata-rata loss dari berbagai batch yang diproses selama setiap epoch.